In [4]:
import pandas as pd

df = pd.read_csv('data/train_data_categories.csv')
df = df[df['video_id'] != 'b4d70f82038d1d97f1b3ce2a493d12c8']

def get_first_level(x):
    return x.split(':')[0].strip()

def get_second_level(x):
    if x.count(':') > 0:
        return x.split(':')[1].strip()
    else:
        return '-'
    
def get_third_level(x):
    if x.count(':') == 2:
        return x.split(':')[2].strip()
    else:
        return '-'

df['first_level'] = df['tags'].apply(get_first_level)
df['second_level'] = df['tags'].apply(get_second_level)
df['third_level'] = df['tags'].apply(get_third_level)

In [5]:
df_cat = pd.read_csv('data/IAB_tags.csv')[:-1]

', '.join(df_cat['Уровень 1 (iab)'].unique().tolist())

'Транспорт, Книги и литература, Бизнес и финансы, Карьера, Образование, События и достопримечательности, Семья и отношения, Изобразительное искусство, Еда и напитки, Здоровый образ жизни, Хобби и интересы, Дом и сад, Медицина, Фильмы и анимация, Музыка и аудио, Новости и политика, Личные финансы, Животные, Массовая культура, Недвижимость, Религия и духовность, Наука, Покупки, Спорт, Стиль и красота, Информационные технологии, Телевидение, Путешествия, Игры'

In [6]:
y_test

353    22
985    78
877    64
983    22
31     65
       ..
364    22
274     2
751     3
961    64
69     24
Name: encoded_labels, Length: 420, dtype: int64

In [10]:
import re
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zа-яё0-9\s]', '', text)
    return text

# Подготовка данных
df['text_for_classification'] = df['title']
df['cleaned_title'] = df['text_for_classification'].apply(clean_text)

# Преобразование меток в числовые значения
label_encoder = LabelEncoder()
df['encoded_labels'] = label_encoder.fit_transform(df['first_level'])

X_train, X_test, y_train, y_test = train_test_split(df['cleaned_title'].tolist(), df['encoded_labels'].tolist(), test_size=0.4, random_state=42)

# Использование токенизатора BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Создание кастомного Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Подготовка датасетов
train_dataset = TextDataset(X_train, y_train, tokenizer)
test_dataset = TextDataset(X_test, y_test, tokenizer)

# Определение модели
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=len(label_encoder.classes_))

# Определение аргументов для тренировки
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",

    
)

# Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {
        'accuracy': accuracy_score(p.label_ids, p.predictions.argmax(-1)),
        'f1_micro': f1_score(p.label_ids, p.predictions.argmax(-1), average='micro'),
        'f1_macro': f1_score(p.label_ids, p.predictions.argmax(-1), average='macro'),
    }
)

# Обучение модели
trainer.train()

# Оценка модели
eval_results = trainer.evaluate()

print(f"Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"F1 micro: {eval_results['eval_f1_micro']:.4f}")
print(f"F1 macro: {eval_results['eval_f1_macro']:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/transformers/training_args.py:1951: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


ValueError: Either you do not have an MPS-enabled device on this machine or MacOS version is not 12.3+ or current PyTorch install was not built with MPS enabled.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
import reао


def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zа-яё0-9\s]', '', text)
    return text


df['text_for_classification'] = df['title']
df['cleaned_title'] = df['text_for_classification'].apply(clean_text)

X_train, X_test, y_train, y_test = train_test_split(df['cleaned_title'], df['first_level'], test_size=0.4, random_state=42)

catboost_model = CatBoostClassifier(
    iterations=2000, 
    learning_rate=0.03,
    depth=6,
    verbose=100,
    random_seed=42,
)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1,3),
        max_features=300,
        min_df=5,
    )),
    ('logreg', catboost_model)
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 micro: {f1_micro}')
print(f'F1 macro: {f1_macro}')

0:	learn: 4.1360824	total: 24.6ms	remaining: 49.3s
100:	learn: 2.5534795	total: 2.43s	remaining: 45.8s
200:	learn: 2.2829331	total: 4.85s	remaining: 43.4s
300:	learn: 2.0684874	total: 7.26s	remaining: 41s
400:	learn: 1.9068269	total: 9.63s	remaining: 38.4s
500:	learn: 1.7857777	total: 12s	remaining: 35.9s
600:	learn: 1.6941675	total: 14.3s	remaining: 33.4s
700:	learn: 1.6224169	total: 16.7s	remaining: 31s
800:	learn: 1.5618133	total: 19.1s	remaining: 28.5s
900:	learn: 1.5054654	total: 21.4s	remaining: 26.1s
1000:	learn: 1.4557885	total: 23.8s	remaining: 23.7s
1100:	learn: 1.4120410	total: 26.1s	remaining: 21.3s
1200:	learn: 1.3748276	total: 28.5s	remaining: 19s
1300:	learn: 1.3349474	total: 31s	remaining: 16.6s
1400:	learn: 1.2975616	total: 33.4s	remaining: 14.3s
1500:	learn: 1.2633244	total: 35.8s	remaining: 11.9s
1600:	learn: 1.2315338	total: 38.2s	remaining: 9.52s
1700:	learn: 1.2016943	total: 40.6s	remaining: 7.14s
1800:	learn: 1.1736112	total: 43s	remaining: 4.75s
1900:	learn: 1.1

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
import re

# Функция для очистки текста
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zа-яё0-9\s]', '', text)
    return text

# Применение очистки текста к колонке 'title'
df['cleaned_title'] = df['title'].apply(clean_text)

# Сплит меток по запятой и удаление лишних пробелов
df['labels'] = df['first_level'].apply(lambda x: [label.strip() for label in x.split(',')])

# Преобразование меток в формат, подходящий для многометочного обучения
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['labels'])

X_train, X_test, y_train, y_test = train_test_split(df['cleaned_title'], y, test_size=0.4, random_state=42)

tfidf_params = {
    # 'max_df': 0.9,
    'min_df': 5,
    'ngram_range': (1,2),
    'max_features': 500
}

catboost_model = CatBoostClassifier(
    iterations=1000, 
    learning_rate=0.1,
    depth=6,
    verbose=False,
    random_seed=42,
)

log_reg_model = LogisticRegression()


# Создание пайплайна
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(**tfidf_params)),
    ('catboost', OneVsRestClassifier(catboost_model))
])

# Обучение модели
pipeline.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = pipeline.predict(X_test)

# Оценка точности и F1-меры
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='micro')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')

# Доступ к словарю tfidf и количество слов
tfidf = pipeline.named_steps['tfidf']
vocab_size = len(tfidf.vocabulary_)
print(f'TFIDF Vocabulary Size: {vocab_size}')

/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 4 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 6 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 11 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 28 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 36 is pre

Accuracy: 0.5452
F1 Score: 0.6641
TFIDF Vocabulary Size: 241


In [125]:
for x,y in df['tags'].value_counts().to_dict().items():
    print(x, y)

Массовая культура: Юмор и сатира 224
Религия и духовность: астрология 57
Еда и напитки: Кулинария 41
Массовая культура, Карьера 37
Массовая культура 24
Путешествия, События и достопримечательности: Исторические места и достопримечательности 22
Массовая культура: Юмор и сатира, Семья и отношения 22
Спорт: Рыбалка 20
Дом и сад: Дизайн интерьера 20
Массовая культура, Музыка и аудио 18
Транспорт, Спорт: Автогонки, События и достопримечательности: Спортивные события 17
Еда и напитки: Кулинария, Массовая культура 16
Массовая культура, Спорт 15
Музыка и аудио: Комедия и стендап (Музыка и аудио), События и достопримечательности: Концерты и музыкальные мероприятия 14
Массовая культура: Отношения знаменитостей 13
Религия и духовность: астрология, События и достопримечательности: Комедия и стендап 12
Массовая культура, Хобби и интересы: Декоративно-прикладное искусство 11
Семья и отношения 11
Фильмы и анимация: Семейные и детские фильмы, Фильмы и анимация: Фильмы и анимация  11
Массовая культура,

In [121]:
df[df['first_level'] == 'Массовая культура, Карьера']

,video_id,title,description,tags,first_level,second_level,third_level,cleaned_title,text_for_classification
91,51304cc9ff8e942ce481282b6952dd86,Артмеханика. Концерт группы TIHOTIHO,Концерт группы TIHOTIHO,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,концерт группы tihotiho,Концерт группы TIHOTIHO
196,02b797508b9ffc941887703dfb17d365,Артмеханика. Сезон 2. Выпуск 6. Эволюция веду...,Влад Маленко и Ольга Ершова разбираются в том...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,влад маленко и ольга ершова разбираются в том...,Влад Маленко и Ольга Ершова разбираются в том...
220,034753227cf7772245e3fa97c5274932,"Артмеханика. Игра ""Лицо рекламы"".","Игра ""Лицо рекламы""","Массовая культура, Карьера","Массовая культура, Карьера",-,-,игра лицо рекламы,"Игра ""Лицо рекламы"""
234,13aba71d6501df8db686919affbc9b72,Артмеханика. Интервью. Денис Рогов.,Что такое метавселенная? Где мы можем ощутить ...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,что такое метавселенная где мы можем ощутить к...,Что такое метавселенная? Где мы можем ощутить ...
236,63d9eab25494382081b6ad100efa7ad4,Артмеханика. Концерт группы Dabro.,Все самые лучшие хиты группы Dabro.,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,все самые лучшие хиты группы dabro,Все самые лучшие хиты группы Dabro.
258,d42d194ba6c4e674a6a625106a76fb1e,Артмеханика. Интервью. Степан Липгарт.,Что собой представляет современная архитектура...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,что собой представляет современная архитектура...,Что собой представляет современная архитектура...
259,343470324af6902808606cf1a7503575,Артмеханика. Интервью с сёстрами Набока.,Как сёстры Набока покорили известных актёров и...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,как сёстры набока покорили известных актёров и...,Как сёстры Набока покорили известных актёров и...
261,7413b5707b2119fc392f0ac00cc2a8a1,Артмеханика. Интервью с Анастасией Моргун и Юр...,Зачем сериал Санта-Барбара вернулся к нам спус...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,зачем сериал сантабарбара вернулся к нам спуст...,Зачем сериал Санта-Барбара вернулся к нам спус...
265,9444666b1f0d17c502e46ecbaad08b67,Артмеханика. Сезон 2. Выпуск 15. Подлинное иск...,В нашей студии стало жарко! “Арт Механика” реш...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,в нашей студии стало жарко арт механика решила...,В нашей студии стало жарко! “Арт Механика” реш...
267,c477b6170b5991c1550032c394d93fd0,Артмеханика. Интервью с Евгением Чесом.,Куда приведет тенденция к тотальной кастомизац...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,куда приведет тенденция к тотальной кастомизац...,Куда приведет тенденция к тотальной кастомизац...


In [12]:
from sklearn.metrics import classification_report

print(classification_report(df['first_level'], df_pred['first_level']))

                                                                                       precision    recall  f1-score   support

                                                                    , Стиль и красота       0.00      0.00      0.00         1
                                                                     Бизнес и финансы       0.00      0.00      0.00         6
                                                                            Дом и сад       0.80      0.18      0.30        22
                                                                        Еда и напитки       0.50      0.56      0.53        64
                                                                             Животные       0.00      0.00      0.00         0
                                                                 Здоровый образ жизни       0.00      0.00      0.00         1
                                                                                 Игры       0.12      0.33    

/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.ca